In [13]:
include("spin_chains.jl")
include("vqe.jl")
include("operator.jl")
using LinearAlgebra
using Random

In [14]:
L=4;
Jxy=1.0;
Jz=0.5;
PBCs=true;

In [15]:
xxzmat = xxz_matrix(L,Jxy,Jz,PBCs)
evals = eigvals(xxzmat)
println(evals[1])

-6.744562646538029


In [16]:
xxzham = xxz_model(L,Jxy,Jz,PBCs)

ansatz_ops = map(pauli_string_to_pauli,repeat(["IIXY","XYII","XIYI","XIIY","IXYI"],2));

init_state = zeros(ComplexF64,2^L);
init_state[1] = 1.0;

Random.seed!(1234)
rstate = rand!(zeros(ComplexF64,2^L))
rstate = rstate/sqrt(dot(rstate,rstate))
println(dot(rstate,rstate))

tmp_state = zeros(ComplexF64,2^L);
print("init state energy: ",exp_val(xxzham,init_state,tmp_state),'\n')

optimizer = "LN_COBYLA"
init_pt = ones(Float64,length(ansatz_ops));

function maxiter_callback(hist::ADAPTHistory)
    if length(hist.energy) >= 3
        return true
    else
        return false
    end
end

adapt_results = adapt_vqe(xxzham,ansatz_ops,L,optimizer,Function[maxiter_callback],rstate)

1.0 + 0.0im
init state energy: 2.0 + 0.0im


ADAPTHistory([2.56919, -0.491352, -0.997561], [1.71037, 1.8304, 0.83834], [5, 8, 1], Array{Float64,1}[[1.10301, 1.44709, -1.02375, 0.408317, -1.71037, 1.28452, 1.5527, -1.13253, 0.194246, -1.54971], [-0.469641, 0.818803, -1.54549, 1.21002, -0.0984335, -0.288137, 0.725986, -1.8304, 0.907885, 0.137143], [-0.83834, 0.277237, -0.0176813, 0.475119, -0.0694997, -0.710204, 0.356206, -3.93586e-8, 0.324281, -0.0264601]], Array{Float64,1}[[], [], [1.28227], [2.78819, 0.489319]])

In [10]:
adapt_results.energy

3-element Array{Float64,1}:
  2.569187838203773  
 -0.49135241841088745
 -0.49135241841088745

In [11]:
adapt_results.max_grad

3-element Array{Float64,1}:
 5.551115123125783e-17
 5.551115123125783e-17
 3.469446951953614e-18

In [12]:
adapt_results.max_grad_ind

3-element Array{Int64,1}:
 5
 5
 4

In [ ]:
println("energies: ",adapt_results.energy)
# println("energy difference exact - vqe = ",evals[1]-vqe_results[1])

In [13]:
println(xxzham,'\n',ansatz_ops,'\n',opt,'\n', init_pt,'\n', L,'\n', init_state)
vqe_results = VQE(xxzham,ansatz_ops,opt,init_pt,L,init_state)

Operator(Pauli[Pauli(x=1100, y=0000, z=0000, ph=+1), Pauli(x=0000, y=1100, z=0000, ph=+1), Pauli(x=0000, y=0000, z=1100, ph=+1), Pauli(x=110, y=000, z=000, ph=+1), Pauli(x=000, y=110, z=000, ph=+1), Pauli(x=000, y=000, z=110, ph=+1), Pauli(x=11, y=00, z=00, ph=+1), Pauli(x=00, y=11, z=00, ph=+1), Pauli(x=00, y=00, z=11, ph=+1), Pauli(x=1001, y=0000, z=0000, ph=+1), Pauli(x=0000, y=1001, z=0000, ph=+1), Pauli(x=0000, y=0000, z=1001, ph=+1)], Complex{Float64}[1.0+0.0im, 1.0+0.0im, 0.5+0.0im, 1.0+0.0im, 1.0+0.0im, 0.5+0.0im, 1.0+0.0im, 1.0+0.0im, 0.5+0.0im, 1.0+0.0im, 1.0+0.0im, 0.5+0.0im])
Pauli{UInt64}[Pauli(x=10, y=01, z=00, ph=+1), Pauli(x=1000, y=0100, z=0000, ph=+1), Pauli(x=1000, y=0010, z=0000, ph=+1), Pauli(x=1000, y=0001, z=0000, ph=+1), Pauli(x=100, y=010, z=000, ph=+1), Pauli(x=10, y=01, z=00, ph=+1), Pauli(x=1000, y=0100, z=0000, ph=+1), Pauli(x=1000, y=0010, z=0000, ph=+1), Pauli(x=1000, y=0001, z=0000, ph=+1), Pauli(x=100, y=010, z=000, ph=+1)]
Opt(LN_COBYLA, 10)
[1.0, 1.0,

(-6.744562646537241, [2.95962, 2.49109, 1.52576, 0.82503, 1.0931, 1.10728, 0.467375, 0.7854, 0.459662, 0.87976], :ROUNDOFF_LIMITED)

In [38]:
for op in ansatz_ops
    com=commutator(xxzham,Operator([op],[1.0]))
    println(exp_val(blah,init_state,tmp_state))
end

0.0 - 4.440892098500626e-16im
0.0 - 4.440892098500626e-16im
0.0 - 4.440892098500626e-16im
0.0 - 4.440892098500626e-16im
0.0 - 4.440892098500626e-16im
0.0 - 4.440892098500626e-16im
0.0 - 4.440892098500626e-16im
0.0 - 4.440892098500626e-16im
0.0 - 4.440892098500626e-16im
0.0 - 4.440892098500626e-16im


In [59]:
p_op = pauli_string_to_pauli("XXXX")
println(p_op)
com=commutator(xxzham,Operator([p_op],[1.0]))
grad = exp_val(blah,init_state,tmp_state)
print("XXXX",' ',grad,'\n')

Pauli(x=1111, y=0000, z=0000, ph=+1)
XXXX 0.0 - 4.440892098500626e-16im


In [80]:
pvals = "IXYZ"
allplus = (1/sqrt(2^L))*ones(ComplexF64,2^L)
Random.seed!(1234)
rstate = rand!(zeros(ComplexF64,2^L))
rstate = rstate/sqrt(dot(rstate,rstate))
for (a,b,c,d) in Iterators.product(1:4,1:4,1:4,1:4)
    pstr = pvals[a]*pvals[b]*pvals[c]*pvals[d]
    p_op = pauli_string_to_pauli(pstr)
    com=commutator(xxzham,Operator([p_op],[1.0]))
    grad = exp_val(blah,rstate,tmp_state)
    print(pstr,' ',grad,'\n')
end

IIII 5.551115123125783e-17 + 0.9826902085389825im
XIII 5.551115123125783e-17 + 0.9826902085389825im
YIII 5.551115123125783e-17 + 0.9826902085389825im
ZIII 5.551115123125783e-17 + 0.9826902085389825im
IXII 5.551115123125783e-17 + 0.9826902085389825im
XXII 5.551115123125783e-17 + 0.9826902085389825im
YXII 5.551115123125783e-17 + 0.9826902085389825im
ZXII 5.551115123125783e-17 + 0.9826902085389825im
IYII 5.551115123125783e-17 + 0.9826902085389825im
XYII 5.551115123125783e-17 + 0.9826902085389825im
YYII 5.551115123125783e-17 + 0.9826902085389825im
ZYII 5.551115123125783e-17 + 0.9826902085389825im
IZII 5.551115123125783e-17 + 0.9826902085389825im
XZII 5.551115123125783e-17 + 0.9826902085389825im
YZII 5.551115123125783e-17 + 0.9826902085389825im
ZZII 5.551115123125783e-17 + 0.9826902085389825im
IIXI 5.551115123125783e-17 + 0.9826902085389825im
XIXI 5.551115123125783e-17 + 0.9826902085389825im
YIXI 5.551115123125783e-17 + 0.9826902085389825im
ZIXI 5.551115123125783e-17 + 0.9826902085389825im
